In [1]:
import numpy as np

In [2]:
def fit(x_train,y_train):
    res = {}
    classes = set(y_train)
    res["total_data"] = len(y_train)
    for curr_class in classes:
        res[curr_class]= {}
        num_features = x_train.shape[1]
        curr_class_rows = (y_train==curr_class)
        x_train_curr = x_train[curr_class_rows]
        y_train_curr = y_train[curr_class_rows]
        res[curr_class]["total_count"] =len(y_train_curr)
#         res['total_data']+=len(y_train_curr)
        
        for j in range(1,num_features+1):
                res[curr_class][j] = {}
                values = set(x_train[:,j-1])
                for curr_val in values:
                    res[curr_class][j][curr_val] = (x_train_curr[:,j-1]==curr_val).sum()
                    
    return res

In [36]:
def probability(curr_class,dictionary,x):
    num_features = len(dictionary[curr_class].keys()) -1
    prob  = np.log(dictionary[curr_class]['total_count'])- np.log(dictionary['total_data'])
    
    for j in range(1,num_features +1):
        prob += np.log(dictionary[curr_class][j][x[j-1]] + 1) - np.log(dictionary[curr_class]["total_count"]+ len(dictionary[curr_class][j].keys()))
    return prob

In [26]:
def predictSinglePoint(x,dictionary):
    max_prob = 0 
    max_class = 0
    first_run= True
    total_classes = dictionary.keys()
    
    for curr_class in total_classes:
        if curr_class== 'total_data':
            continue
        prob = probability(curr_class,dictionary,x)
        if(first_run or prob>max_prob):
            max_prob = prob
            max_class = curr_class
        first_run = False

    return max_class

In [6]:
def predict(x_test,dictionary):
    y_test = np.zeros(len(x_test),int)
    for i in range(len(x_test)):
        y_test[i] = predictSinglePoint(x_test[i],dictionary)

    return y_test

In [7]:
def make_labelled(column):
    second_limit = column.mean()
    first_limit = (1/2) * second_limit
    third_limit = 1.5 * second_limit
    for i in range(len(column)):
        if column[i]<first_limit:
            column[i] = 0
        elif column[i]<second_limit:
            column[i] = 1
        elif column[i]<third_limit:
            column[i] = 2
        else:
            column[i] = 3
    return column

In [43]:
from sklearn import datasets
iris = datasets.load_iris()
x = iris.data
y = iris.target

for i in range(x.shape[-1]):
    x[:,i] = make_labelled(x[:,i])

from sklearn import model_selection

x_train,x_test,y_train,y_test = model_selection.train_test_split(x,y,random_state = 0,test_size = 0.32)

dictionary = fit(x_train,y_train)
y_pred = predict(x_test,dictionary)
dictionary['total_data']
# print(dictionary)
print(y_pred)

[2 1 0 2 0 2 0 1 1 1 1 1 1 1 1 0 1 1 0 0 2 1 0 0 2 0 0 1 1 0 2 1 0 2 2 1 0
 1 1 1 2 0 2 0 0 1 2 2]


In [44]:
from sklearn.metrics import classification_report , confusion_matrix
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1       0.95      1.00      0.97        19
           2       1.00      0.92      0.96        13

    accuracy                           0.98        48
   macro avg       0.98      0.97      0.98        48
weighted avg       0.98      0.98      0.98        48

[[16  0  0]
 [ 0 19  0]
 [ 0  1 12]]


In [47]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(x_train,y_train)
y_pred = clf.predict(x_test)
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

              precision    recall  f1-score   support

           0       1.00      0.81      0.90        16
           1       0.40      0.11      0.17        19
           2       0.43      1.00      0.60        13

    accuracy                           0.58        48
   macro avg       0.61      0.64      0.56        48
weighted avg       0.61      0.58      0.53        48

[[13  3  0]
 [ 0  2 17]
 [ 0  0 13]]
